<font color="Lime"><h1 align ='center'><b><u>Funciones de costo frente a métricas de evaluación</u></b></h1></font>

En las últimas unidades, hemos empezado a ver una división en las funciones de costo (que enseña el modelo) y las métricas de evaluación, que es cómo evaluamos el modelo nosotros mismos.

## <font color='B8E139'><b>Todas las funciones de costo pueden ser métricas de evaluación</b></font>

Todas las funciones de costo pueden ser métricas de evaluación, aunque no necesariamente intuitivas. Como ejemplo, la pérdida de registros. Los valores no son intuitivos.


## <font color='B8E139'><b>Algunas métricas de evaluación no pueden ser funciones de costo.</b></font>

- Es difícil que algunas métricas de evaluación se conviertan en funciones de costo.
- Esto se debe a restricciones prácticas y matemáticas.
- A veces, las cosas no son fáciles de calcular (por ejemplo, "lo perruno que es algo").
- Las funciones de costo son idealmente suaves. Por ejemplo, la precisión es útil, pero si cambiamos ligeramente el modelo, no se advertirá. Dado que el ajuste es un procedimiento con muchos cambios pequeños, da la impresión de que las modificaciones no llevarán a la mejora.
- Gráfico de función de costo con una gran cantidad de bits planos
- Actualice las curvas ROC de antes. Para ello, es necesario cambiar el umbral a todo tipo de valores, pero al final del día, nuestro modelo solo tendrá uno (0,5).

<div>
<p align="center">
<img src="https://raw.githubusercontent.com/JuanCamilo18/MM_Datos/main/imags/MCyDD/MC_03_01.JPG">
</p>
</div>

## <font color='B8E139'><b>No todo es malo.</b></font>

Puede ser frustrante descubrir que no podemos utilizar nuestra métrica favorita como función de costo. Sin embargo, hay un aspecto positivo, que está relacionado con el hecho de que todas las métricas son simplificaciones de lo que queremos conseguir; ninguna es perfecta. Lo que esto significa es que los modelos complejos a menudo "hacen trampa": encuentran una manera de obtener costos bajos sin encontrar realmente una regla general que resuelva nuestro problema. Tener una métrica que no actúe como función de costo nos permite comprobar que el modelo no ha encontrado la forma de hacer trampas. Si sabemos que un modelo está tomando atajos, podemos replantear nuestra estrategia de entrenamiento.

Ya hemos visto este "engaño" varias veces. Por ejemplo, cuando los modelos sobreajustan los datos de entrenamiento, básicamente están "memorizando" las respuestas correctas en lugar de encontrar una regla general que podamos aplicar con éxito a otros datos. Utilizamos conjuntos de datos de prueba como "prueba de congruencia" para comprobar que el modelo no acaba de hacer esto. También hemos visto que, con datos desequilibrados, los modelos pueden aprender a dar siempre la misma respuesta (por ejemplo, "false") sin tener en cuenta las características, porque, por término medio, esto es correcto y proporciona un pequeño error.

Los modelos complejos también encuentran atajos de otras maneras. A veces, los modelos complejos pueden sobreajustar la propia función de costo. Por ejemplo, imaginemos que intentamos construir un modelo que pueda dibujar perros. Tenemos una función de costo que comprueba que la imagen es marrón, muestra una textura peluda y contiene un objeto del tamaño adecuado. Con esta función de costo, un modelo complejo podría aprender a crear una bola de pelo marrón, no porque se parezca a un perro, sino porque proporciona un costo bajo y es fácil de generar. Si tenemos una métrica externa que cuente el número de patas y cabezas (lo que no se puede utilizar fácilmente como función de costo porque no son métricas suaves), nos daremos cuenta rápidamente de si nuestro modelo está haciendo trampas, y nos replantearemos cómo lo estamos entrenando. Por el contrario, si nuestra métrica alternativa obtiene una buena puntuación, podemos tener cierta confianza en que el modelo ha captado la idea de cómo debe ser un perro, en lugar de limitarse a engañar a la función de costo para obtener un valor bajo.



---



## <font color='B8E139'><b>Ejercicio: Varias métricas y curvas ROC</b></font>


## <font color='B8E139'><b>Ejercicio: Más métricas derivadas de matrices de confusión</b></font>

En este ejercicio aprenderemos sobre diferentes métricas, usándolas para explicar los resultados obtenidos del modelo de clasificación binaria que construimos en la unidad anterior.

### <font color='B8E139'><b>Visualización de datos</b></font>

Usaremos el conjunto de datos con diferentes clases de objetos encontrados en la montaña una vez más:

In [1]:
import pandas
import numpy
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/snow_objects.csv

#Import the data from the .csv file
dataset = pandas.read_csv('snow_objects.csv', delimiter="\t")

#Let's have a look at the data
dataset

--2023-03-26 06:40:14--  https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21511 (21K) [text/plain]
Saving to: ‘graphing.py’

graphing.py         100%[===================>]  21.01K  --.-KB/s    in 0.002s  

2023-03-26 06:40:14 (11.4 MB/s) - ‘graphing.py’ saved [21511/21511]

--2023-03-26 06:40:14--  https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/snow_objects.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awa

,size,roughness,color,motion,label
0,50.959361,1.318226,green,0.054290,tree
1,60.008521,0.554291,brown,0.000000,tree
2,20.530772,1.097752,white,1.380464,tree
3,28.092138,0.966482,grey,0.650528,tree
4,48.344211,0.799093,grey,0.000000,tree
...,...,...,...,...,...
2195,1.918175,1.182234,white,0.000000,animal
2196,1.000694,1.332152,black,4.041097,animal
2197,2.331485,0.734561,brown,0.961486,animal
2198,1.786560,0.707935,black,0.000000,animal



Recuerde que para usar el conjunto de datos anterior para la clasificación binaria , debemos agregar otra columna al conjunto de datos y configurarla True donde está la etiqueta original hiker y False dónde no.

Luego agreguemos esa etiqueta, dividamos el conjunto de datos y entrenemos el modelo nuevamente:

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Add a new label with true/false values to our dataset
dataset["is_hiker"] = dataset.label == "hiker"

# Split the dataset in an 70/30 train/test ratio. 
train, test = train_test_split(dataset, test_size=0.3, random_state=1, shuffle=True)

# define a random forest model
model = RandomForestClassifier(n_estimators=1, random_state=1, verbose=False)

# Define which features are to be used 
features = ["size", "roughness", "motion"]

# Train the model using the binary label
model.fit(train[features], train.is_hiker)

print("Model trained!")

Model trained!



Ahora podemos usar este modelo para predecir si los objetos en la nieve son excursionistas o no.

Tracemos su matriz de confusión :

In [3]:
# sklearn has a very convenient utility to build confusion matrices
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

# Calculate the model's accuracy on the TEST set
actual = test.is_hiker
predictions = model.predict(test[features])

# Build and print our confusion matrix, using the actual values and predictions 
# from the test set, calculated in previous cells
cm = confusion_matrix(actual, predictions, normalize=None)

# Create the list of unique labels in the test set, to use in our plot
# I.e., ['True', 'False',]
unique_targets = sorted(list(test["is_hiker"].unique()))

# Convert values to lower case so the plot code can count the outcomes
x = y = [str(s).lower() for s in unique_targets]

# Plot the matrix above as a heatmap with annotations (values) in its cells
fig = ff.create_annotated_heatmap(cm, x, y)

# Set titles and ordering
fig.update_layout(  title_text="<b>Confusion matrix</b>", 
                    yaxis = dict(categoryorder = "category descending"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted label",
                        xref="paper",
                        yref="paper"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.15,
                        y=0.5,
                        showarrow=False,
                        text="Actual label",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# We need margins so the titles fit
fig.update_layout(margin=dict(t=80, r=20, l=120, b=50))
fig['data'][0]['showscale'] = True
fig.show()

In [4]:
# Let's also calculate some values that will be used throughout this exercise
# We already have actual values and corresponding predictions, defined above
correct = actual == predictions
tp = numpy.sum(correct & actual)
tn = numpy.sum(correct & numpy.logical_not(actual))
fp = numpy.sum(numpy.logical_not(correct) & actual)
fn = numpy.sum(numpy.logical_not(correct) & numpy.logical_not(actual))

print("TP - True Positives: ", tp)
print("TN - True Negatives: ", tn)
print("FP - False positives: ", fp)
print("FN - False negatives: ", fn)

TP - True Positives:  75
TN - True Negatives:  523
FP - False positives:  29
FN - False negatives:  33



Podemos usar los valores y la matriz anteriores para ayudarnos a comprender otras métricas.

## <font color='B8E139'><b>Cálculo de métricas</b></font>

De aquí en adelante, analizaremos más de cerca cada una de las siguientes métricas, cómo se calculan y cómo pueden ayudar a explicar nuestro modelo actual.

- Exactitud
- Sensibilidad/recuperación
- especificidad
- Precisión
- Tasa de falsos positivos

Primero recordemos algunos términos útiles:

- TP = Verdaderos positivos: una etiqueta positiva se predice correctamente
- TN = Verdaderos negativos: una etiqueta negativa se predice correctamente
- FP = Falsos positivos: una etiqueta negativa se predice como positiva
- FN = Falsos negativos: una etiqueta positiva se predice como negativa

### <font color='B8E139'><b>Exactitud</b></font>
Exactitud
La precisión es el número de predicciones correctas dividido por el número total de predicciones:

```
    accuracy = (TP+TN) / number of samples
```

Posiblemente sea la métrica más básica utilizada pero, como hemos visto, no es la más fiable cuando se utilizan conjuntos de datos desequilibrados .

En codigo:

In [5]:
# Calculate accuracy
# len(actual) is the number of samples in the set that generated TP and TN
accuracy = (tp+tn) / len(actual) 

# print result as a percentage
print(f"Model accuracy is {accuracy:.2f}%")

Model accuracy is 0.91%


### <font color='B8E139'><b>Sensibilidad/recuperación</b></font>

Sensitivity y Recall son nombres intercambiables para la misma métrica, que expresa la fracción de muestras predichas correctamente por un modelo:

```
sensitivity = recall = TP / (TP + FN)
```

Esta es una métrica importante, que nos dice cómo, de todas las muestras realmente positivas , cuántas se predicen correctamente como positivas.

En codigo:

In [6]:
# code for sensitivity/recall
sensitivity = recall = tp / (tp + fn)

# print result as a percentage
print(f"Model sensitivity/recall is {sensitivity:.2f}%")

Model sensitivity/recall is 0.69%


### <font color='B8E139'><b>Especificidad</b></font>

La especificidad expresa la fracción de etiquetas negativas predichas correctamente sobre el número total de muestras negativas existentes:

```
specificity = TN / (TN + FP)
```

Nos dice cómo, de todas las muestras realmente negativas , cuántas se predicen correctamente como negativas

Se puede calcular con el siguiente código:

In [7]:
# Code for specificity
specificity = tn / (tn + fp)

# print result as a percentage
print(f"Model specificity is {specificity:.2f}%")

Model specificity is 0.95%


### <font color='B8E139'><b>Precisión</b></font>

La precisión expresa la proporción de muestras positivas pronosticadas correctamente sobre todas las predicciones positivas:

```
precision = TP / (TP + FP)
```

En otras palabras, indica cómo de todas las predicciones positivas, cuántas son etiquetas verdaderamente positivas.

Se puede calcular con el siguiente código:

In [8]:
# Code for precision

precision = tp / (tp + fp)

# print result as a percentage
print(f"Model precision is {precision:.2f}%")

Model precision is 0.72%


### <font color='B8E139'><b>Tasa de falsos positivos</b></font>

Tasa de falsos positivos o FPR, es el número de predicciones positivas incorrectas dividido por el número total de muestras negativas:

```
false_positive_rate = FP / (FP + TN)
```

De todos los negativos reales, ¿cuántos se clasificaron erróneamente como positivos?

En codigo:

In [9]:
# Code for false positive rate
false_positive_rate = fp / (fp + tn)

# print result as a percentage
print(f"Model false positive rate is {false_positive_rate:.2f}%")

Model false positive rate is 0.05%



Observe que la suma de `specificity` y `false positive rate` siempre debe ser igual a 1.

### **Conclusión**

Hay varias métricas diferentes que pueden ayudarnos a evaluar el rendimiento de un modelo, en el contexto de la calidad de sus predicciones.

La elección de las métricas más adecuadas, sin embargo, es principalmente una función de los datos y del problema que estamos tratando de resolver.


**Resumen**

Cubrimos los siguientes temas en esta unidad:

- Cómo calcular las medidas muy básicas utilizadas en la evaluación de modelos de clasificación: TP, FP, TN, FN.
- Cómo usar las medidas anteriores para calcular métricas más significativas, como:
  - Exactitud
  - Sensibilidad/recuperación
  - especificidad
  - Precisión
  - Tasa de falsos positivos

- Cómo la elección de las métricas depende del conjunto de datos y del problema que estamos tratando de resolver.